<a href="https://colab.research.google.com/github/ashutosh3060/friday-burger-mojito/blob/master/data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Contents

0. Libraries
1. User-Defined Functions
2. Import Data and Basic Understanding
3. Data Quality Checks   
  3.1Duplicate    
        3.1.1 Complete row duplicate
        3.1.2 Unique Id duplicate
  3.2 Missing value
4. Feature Engineering
5. Save Final Dataset with new features

## 0. Libraries 

In [1]:
# warnings
import warnings
warnings.filterwarnings("ignore")

# Numpy, Pandas
import numpy as np
import pandas as pd

# Display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 15)

## 1. User-defined Functions

In [15]:
def null_perc_check (df):
    '''
    Calculates missing value count and percentage for all the columns in a dataframe

    Inputs
    -------
    df : dataframe
        The dataframe for which missing value distribution needs to checked

    Output
    -------
    dataframe
        a dataframe showing missing value count and percentage for all the columns
    '''
    missing_value_df = pd.DataFrame(index = df.keys(), data =df.isnull().sum(), columns = ['Missing_Value_Count'])
    missing_value_df['Missing_Value_Percentage'] = np.round(((df.isnull().mean())*100),1)
    sorted_df = missing_value_df.sort_values('Missing_Value_Count',ascending= False)
    return sorted_df

## 2. Import Data and Basic Understanding

In [3]:
# Define the path, filename

order_path = "/content/"
order_file = "machine_learning_challenge_order_data.csv"
label_path = "/content/"
label_file = "machine_learning_challenge_labeled_data.csv"

In [4]:
# import the dataset as a dataframe

df_order = pd.read_csv(order_path+order_file)
df_label = pd.read_csv(label_path+label_file)

In [7]:
# df_order: shape and first few records

print(df_order.shape)
df_order.head(3)

(786600, 13)


,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324


In [8]:
# df_label: shape and first few records

print(df_label.shape)
df_label.head(3)

(245455, 2)


,customer_id,is_returning_customer
0,000097eabfd9,0
1,0000e2c6d9be,0
2,000133bb597f,1


In [9]:
# Number of unique customers in both the datasets

print(f'df_order: {df_order.customer_id.nunique()}')
print(f'df_label: {df_label.customer_id.nunique()}')

df_order: 245455
df_label: 245455


* As mentioned in the problem document, it seems like multiple records are present for a single customer (Historical data)

* In both order and label datasets, #unique customers are same.

* Before merging both the datasets, need to check the intersection of the unique customer_id

In [10]:
# Intersection of unique customers in both the datasets

cust_intersect = pd.merge(df_order["customer_id"], df_label["customer_id"], how='inner', on=['customer_id'])
print(f'Number of common unique customers in both the datasets: {cust_intersect["customer_id"].nunique()}')

Number of common unique customers in both the datasets: 245455


* This confirms, all the unique customer_id in both the datasets are same.

* So, I will try to extract all the features by grouping, aggregating and other thought process. And then will merge the target variable from the label dataset

* For now, I can make the df_order as the primary dataset for independent feature extraction.

In [11]:
# Datatypes overview of the order dataset

df = df_order.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786600 entries, 0 to 786599
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   customer_id          786600 non-null  object 
 1   order_date           786600 non-null  object 
 2   order_hour           786600 non-null  int64  
 3   customer_order_rank  761833 non-null  float64
 4   is_failed            786600 non-null  int64  
 5   voucher_amount       786600 non-null  float64
 6   delivery_fee         786600 non-null  float64
 7   amount_paid          786600 non-null  float64
 8   restaurant_id        786600 non-null  int64  
 9   city_id              786600 non-null  int64  
 10  payment_id           786600 non-null  int64  
 11  platform_id          786600 non-null  int64  
 12  transmission_id      786600 non-null  int64  
dtypes: float64(4), int64(7), object(2)
memory usage: 78.0+ MB


* Only **customer_order_rank**  has missing values.
* order_date is object, datatype casting is needed for operation.

In [21]:
# Datatypes overview of the customer label dataset

df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245455 entries, 0 to 245454
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   customer_id            245455 non-null  object
 1   is_returning_customer  245455 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


* No missing values
* Also the datatype seems okay.

## 3. Data Quality Checks

### 3.1 Duplicate record checks

#### 3.1.1 Complete Row Duplicate

In [12]:
# Check duplicates

check_dup = df.duplicated(subset=None, keep='first')
print(f'Number of Total records:  {df.shape[0]}')
print(f'Number of Unique records: {check_dup.shape[0]}')
print(f'Number of Duplicates: {df.shape[0] - check_dup.shape[0]}')

Number of Total records:  786600
Number of Unique records: 786600
Number of Duplicates: 0


* Both the total and unique records are same, so there are no duplicate present.

#### 3.1.2 customer_id (unique identifier) Duplicate

In [13]:
# Number of records for each customer_id

df["customer_id"].value_counts()

15edce943edd    386
8745a335e9cf    288
d956116d863d    286
0063666607bb    273
ae60dce05485    270
               ... 
ba7d5a6e6638      1
4324cbb32eb8      1
9aafd98d48b6      1
bea68ff2ad49      1
ca16c3a1e1af      1
Name: customer_id, Length: 245455, dtype: int64

* As mentioned in the Data Dictionary, a customer_id can appear for multiple times as multiple orders in different dates are present, so a valid business scenario.

### 3.2 Null check - Missing value percentage

In [16]:
# Missing value count and percentage check

null_perc_check(df)

,Missing_Value_Count,Missing_Value_Percentage
customer_order_rank,24767,3.1
customer_id,0,0.0
order_date,0,0.0
order_hour,0,0.0
is_failed,0,0.0
voucher_amount,0,0.0
delivery_fee,0,0.0
amount_paid,0,0.0
restaurant_id,0,0.0
city_id,0,0.0


* There are no missing values except for customer_order_rank.
* customer_order_rank can be null whenever an order is failed.
* Once this condition is checked, we can confirm the actual missing percentage of customer_order_rank

In [18]:
# Check number of failed orders and the customer_order_rank for the same

print(f'#Failed orders: {df.loc[df["is_failed"]==1].shape[0]}')
print(f'unique customer_rank_order values:  {df.loc[df["is_failed"]==1]["customer_order_rank"].unique()}')

#Failed orders: 24767
unique customer_rank_order values:  [nan]


1. #Failed orders = #Missing values for customer_order_rank
2. For all failed orders, customer_order_rank is null    
* Above 2 statements prove that the Nulls are business case wise correct; hence these are not semantically missing values.

In [19]:
# Check the missing value percentage for label dataset

null_perc_check(df_label)

,Missing_Value_Count,Missing_Value_Percentage
customer_id,0,0.0
is_returning_customer,0,0.0


* There are no missing values for customer label dataset.

## 4. Feature Engineering

In [22]:
df.columns

Index(['customer_id', 'order_date', 'order_hour', 'customer_order_rank',
       'is_failed', 'voucher_amount', 'delivery_fee', 'amount_paid',
       'restaurant_id', 'city_id', 'payment_id', 'platform_id',
       'transmission_id'],
      dtype='object')

In [30]:
df.tail(5)

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
786595,fffe9d5a8d41,2016-09-30,20,NaN,1,0.0,0.0,10.7262,983498,10346,1779,29463,212
786596,ffff347c3cfa,2016-08-17,21,1.0,0,0.0,0.0,7.5933,52893498,41978,1619,30359,4356
786597,ffff347c3cfa,2016-09-15,21,2.0,0,0.0,0.0,5.9472,164653498,41978,1619,30359,4356
786598,ffff4519b52d,2016-04-02,19,1.0,0,0.0,0.0,21.7710,16363498,80562,1491,29751,4228
786599,ffffccbfc8a4,2015-05-30,20,1.0,0,0.0,0.0,16.4610,150293498,45952,1619,29463,4324


### List of features to be created

**1. How old the customer in business according to order_date**   
i.e. need to extract customer's first order date within this 2 years time span   
**2. From how many days Customer has not ordered**   
i.e. Last order before how many days   
**3. Customer ordered for a period of n days (continuously)**   
i.e. Difference between first_order_date and last_order_date    
**4. Common order hour**   
i.e. Most occuring / mode of order_hour   
**5. Number of successful orders**   
i.e. count of customer_order_rank (need to exclude null values)   
**6. Number of failed orders**   
i.e. count of is_failed==1 
**7. Average voucher amount used**      
i.e. Mean of voucher amount    
**8. Average delivery fee paid**    
i.e. Mean of delivery_fee       
**9. Average amount paid**    
i.e. Mean of voucher amount   
**10. Customer has ordered from how many unique number of restaurants**     
i.e. # unique restaurant_id         
**11. Customer has ordered from how many unique number of cities**     
i.e. # unique city_id     
**12. Customer has used how many unique number of payment_id**     
i.e. # unique payment_id     
**13. Customer has ordered from how many unique number of platforms**     
i.e. # unique platform_id         
**14. Customer has used how many unique number of transmission_id**  

In [31]:
df.loc[df["customer_id"]=="fffe9d5a8d41"]

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
786593,fffe9d5a8d41,2016-07-31,21,NaN,1,0.0,0.0,8.4429,156133498,10346,1811,29463,212
786594,fffe9d5a8d41,2016-09-30,20,1.0,0,0.0,0.0,10.7262,983498,10346,1779,29463,4228
786595,fffe9d5a8d41,2016-09-30,20,NaN,1,0.0,0.0,10.7262,983498,10346,1779,29463,212


* Need to think, how to process the failed and successful orders